In [12]:
import torch
import time
import yaml
import os
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
from ultralytics import YOLO

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# 데이터셋 경로 설정 (yolo는 dataset 경로를 images와 labels로 해주어야 인식이 가능)
data_root = 'D:\\Falldown\\Dataset\\Resized_Dataset'
train_root = f'{data_root}\\Train\\images'
val_root = f'{data_root}\\Val\\images'
test_root = f'{data_root}\\Test\\images'

# 클래스 설정
class_names = {0 : 'Human'} # 팀원 yaml에서는 0을 비낙상, 1을 낙상으로 클래스 설정함
num_classes = len(class_names)

# yaml 설정
yaml_info = {
    'path' : r'D:\Falldown', # YOLOv8 기본 경로
    'names': class_names,
    'nc': num_classes,
    'train': train_root,
    'val': val_root,
    'test': test_root
}

# YAML 파일 저장 경로
yaml_file_path = 'D:\\Falldown\\yaml_info_yolov8s.yaml'

# YAML 파일 생성 또는 확인
if os.path.exists(yaml_file_path):
    print(f"YAML 파일이 이미 존재합니다: {yaml_file_path}")
else:
    with open(yaml_file_path, 'w') as f:
        yaml.dump(yaml_info, f)
    print(f"YAML 파일이 생성되었습니다: {yaml_file_path}")

YAML 파일이 이미 존재합니다: D:\Falldown\yaml_info_yolov8s.yaml


In [4]:
model = YOLO('yolov8s.pt')

In [5]:
# 학습 디렉토리 이름 설정
training_name = 'human_fall_s'
training_dir = f"D:\\Falldown\\code-git\\runs\\detect\\{training_name}"

try:
    if os.path.exists(training_dir):
        print(f"학습이 이미 완료된 디렉토리가 존재합니다: {training_dir}")
    else:
        start_time = time.time()

        # 학습 실행
        result = model.train(
            data='D:\\Falldown\\yaml_info_yolov8s.yaml',
            epochs=50,
            batch=16,
            imgsz=640,
            device=device,
            workers=12,
            amp=True,
            patience=30,
            name=training_name
        )

        end_time = time.time()
        execution_time = end_time - start_time
        print(f"학습 실행 시간: {execution_time:.4f} 초")  # 약 20시간 소요

except KeyboardInterrupt:
    print("KeyboardInterrupt: 사용자가 학습을 중단하였습니다.")

학습이 이미 완료된 디렉토리가 존재합니다: D:\Falldown\code-git\runs\detect\human_fall_s


In [8]:
# 검증 시작 시간 기록
start_time = time.time()
model_path = f'{training_dir}\\weights\\best.pt'

# YOLO 모델 로드 및 검증 실행
model = YOLO(model_path)
val_results = model.val(
    data='D:\\Falldown\\yaml_info_yolov8s.yaml',
    imgsz=640,
    batch=32,
    device=device
)

# 평가 종료 시간 기록
end_time = time.time()
execution_time = end_time - start_time

# 결과 출력
print(f"Validation Results:")
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"실행 시간: {execution_time:.4f} 초")

Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning D:\Falldown\Dataset\Resized_Dataset\Val\labels.cache... 9440 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9440/9440 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 295/295 [01:22<00:00,  3.58it/s]


                   all       9440       9440      0.982      0.661      0.827       0.81
              Non_Fall       9440       9440      0.982      0.661      0.827       0.81
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val4
Validation Results:
mAP50: 0.8267
mAP50-95: 0.8099
실행 시간: 116.8889 초


In [ ]:
# 테스트 시작 시간 기록
start_time = time.time()

# YOLO 모델 로드 및 test 실행
model = YOLO(model_path)
test_results = model.val(
    data='D:\\Falldown\\yaml_info_yolov8s.yaml',
    imgsz=640,
    batch=32,
    device=device,
    split="test"  # Test 데이터셋으로 평가
)

# 테스트 종료 시간 기록
end_time = time.time()
execution_time = end_time - start_time

# 테스트 결과 출력
print(f"Test Results:")
print(f"mAP50: {test_results.box.map50:.4f}")
print(f"mAP50-95: {test_results.box.map:.4f}")
print(f"실행 시간: {execution_time:.4f} 초")

val: Scanning D:\Falldown\Dataset\Resized_Dataset\Test\labels... 9580 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9580/9580 [00:04<00:00, 2359.04it/s]


val: New cache created: D:\Falldown\Dataset\Resized_Dataset\Test\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 300/300 [01:24<00:00,  3.56it/s]


                   all       9580       9580      0.985      0.674      0.833      0.815
              Non_Fall       9580       9580      0.985      0.674      0.833      0.815
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val2
Test Results:
mAP50: 0.8332
mAP50-95: 0.8146
실행 시간: 117.0793 초


In [29]:
# 경로 설정
data_root = 'D:\\Falldown\\Dataset\\Resized_Dataset'
test_root = f'{data_root}\\Test\\images'

# YOLO 모델 로드
model_path = "D:\\Falldown\\code-git\\runs\\detect\\human_fall_s\\weights\\best.pt"
model = YOLO(model_path)

# 시각화 함수
def process_and_visualize(image, model, conf=0.5):
    results = model.predict(image, conf=conf, save=False, verbose=False)
    for result in results:
        for box in result.boxes.xyxy.tolist():  # Tensor -> 리스트로 변환
            x1, y1, x2, y2 = map(int, box[:4])
            cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)
    return image

# 랜덤 테스트 이미지 10개 가져오기
image_files = [os.path.join(test_root, file) for file in os.listdir(test_root) if file.endswith('.jpg')]
random.shuffle(image_files)
image_files = image_files[:10]

# 이미지 처리 및 시각화
processed_images = []
for image_file in image_files:
    image = cv2.imread(image_file)
    if image is None:  # 이미지 로드 실패 처리
        print(f"Failed to load {image_file}")
        continue
    processed_image = process_and_visualize(image, model, conf=0.5)
    processed_images.append((os.path.basename(image_file), processed_image))

# Subplot에 시각화된 이미지 추가
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.flatten()

for idx, (image_name, image) in enumerate(processed_images):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    axes[idx].imshow(image_rgb)
    axes[idx].axis('off')
    axes[idx].set_title(image_name)

# 나머지 subplot 빈칸 처리
for ax in axes[len(processed_images):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

<Figure size 2000x800 with 10 Axes>